# Стандартно подключаем датасеты AG-News и imdb

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset_news0 = load_dataset("ag_news")

In [ ]:
count0, count1, count2, count3 = 0, 0, 0, 0
dataset_news = []
for i in range(len(dataset_news0['train'])):
  if dataset_news0['train'][i]['label'] == 0 and count0 < 2000:
    dataset_news.append({'news': dataset_news0['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_news0['train'][i]['label'] == 1 and count1 < 2000:
    dataset_news.append({'news': dataset_news0['train'][i]['text'], 'label': 1})
    count1 += 1
  elif dataset_news0['train'][i]['label'] == 2 and count2 < 2000:
    dataset_news.append({'news': dataset_news0['train'][i]['text'], 'label': 2})
    count2 += 1
  elif dataset_news0['train'][i]['label'] == 3 and count3 < 2000:
    dataset_news.append({'news': dataset_news0['train'][i]['text'], 'label': 3})
    count3 += 1
len(dataset_news)

In [ ]:
import random
random.shuffle(dataset_news)

In [ ]:
# так как потом мне нужно будет использовать и кросс-валидацию, и разделеине на train/test, просто поделю на тексты и метки
dataset_news_texts = [i['news'] for i in dataset_news]
dataset_news_labels = [i['label'] for i in dataset_news]

In [ ]:
dataset_imdb0 = load_dataset("imdb")

count0, count1 = 0, 0
dataset_imdb = []

for i in range(len(dataset_imdb0['train'])):
  if dataset_imdb0['train'][i]['label'] == 0 and count0 < 4000:
    dataset_imdb.append({'text': dataset_imdb0['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_imdb0['train'][i]['label'] == 1 and count1 < 4000:
    dataset_imdb.append({'text': dataset_imdb0['train'][i]['text'], 'label': 1})
    count1 += 1
len(dataset_imdb)

In [ ]:
random.shuffle(dataset_imdb)

In [ ]:
dataset_imdb_texts = [i['text'] for i in dataset_imdb]
dataset_imdb_labels = [i['label'] for i in dataset_imdb]

# Функции предобработки и стемминг/лемматизация

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import re
from nltk.corpus import stopwords

nltk.download('punkt_tab')
nltk.download('stopwords')
stop_words = list(set(stopwords.words("english")))

In [ ]:
def preproccesing(text):
  text = text.lower()
  tokens = word_tokenize(text)
  tokens = [word for word in tokens if not word in stop_words]
  tokens = word_tokenize(re.sub(r'[^a-zA-Zа-яА-Я ]', '', ' '.join(tokens)))  # убираем спец символы, числа и знаки препинания
  return tokens

In [ ]:
# примеры базового preprocesing'а для обоих датасетов
print(dataset_news_texts[0])
print(preproccesing(dataset_news_texts[0]))
print()
print(dataset_imdb_texts[0])
print(preproccesing(dataset_imdb_texts[0]))

In [ ]:
# добавляем стемминг и лемматизацию
nltk.download('wordnet')
nltk.download('omw-1.4')

def stemming(text):
  tokens = preproccesing(text)  # базовый preproccesing
  stemmer = nltk.PorterStemmer()
  stemmed_tokens = [stemmer.stem(token) for token in tokens]  # перебираем токены и применяем алгоритм стемминга
  return stemmed_tokens

In [ ]:
# примеры preproccesing'а после стемминга
print(dataset_news_texts[0])
print(stemming(dataset_news_texts[0]))
print()
print(dataset_imdb_texts[0])
print(stemming(dataset_imdb_texts[0]))

In [ ]:
def lemmatisation(text):
  tokens = preproccesing(text)
  lemma = nltk.WordNetLemmatizer()
  lemma_tokens = [lemma.lemmatize(token) for token in tokens]
  return lemma_tokens

In [ ]:
# примеры preproccesing'а после лемматизации
print(dataset_news_texts[0])
print(lemmatisation(dataset_news_texts[0]))
print()
print(dataset_imdb_texts[0])
print(lemmatisation(dataset_imdb_texts[0]))

# Каждому датасету создадим по 3 варианта: базовый preproccesing, стэмминг и лемматизация

In [ ]:
dataset_news_base = [' '.join(preproccesing(text)) for text in dataset_news_texts]
dataset_news_stem = [' '.join(stemming(text)) for text in dataset_news_texts]
dataset_news_lemma = [' '.join(lemmatisation(text)) for text in dataset_news_texts]

In [ ]:
dataset_imdb_base = [' '.join(preproccesing(text)) for text in dataset_imdb_texts]
dataset_imdb_stem = [' '.join(stemming(text)) for text in dataset_imdb_texts]
dataset_imdb_lemma = [' '.join(lemmatisation(text)) for text in dataset_imdb_texts]

# Создадим два vectorizers: униграммы и униграммы+биграммы

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vect_uni = TfidfVectorizer(ngram_range=(1, 1))
vect_bi = TfidfVectorizer(ngram_range=(1, 2))

# Тестово обучаем RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

rf = RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score

cv_strat_rf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 30],      # Максимальная глубина дерева
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 2, 4],        # Минимальное число образцов в листе
    'class_weight': [None, 'balanced']    # Балансировка классов
}

In [ ]:
grid_search_rf = GridSearchCV(
    rf,
    param_grid_rf,
    cv=cv_strat_rf,
    scoring='f1_weighted',
    n_jobs=-1,
    verbose=2
)

In [ ]:
all_datasets ={
    'news_base': (dataset_news_base, dataset_news_labels),
    'news_stem': (dataset_news_stem, dataset_news_labels),
    'news_lemma': (dataset_news_lemma, dataset_news_labels),
    'imdb_base': (dataset_imdb_base, dataset_imdb_labels),
    'imdb_stem': (dataset_imdb_stem, dataset_imdb_labels),
    'imdb_lemma': (dataset_imdb_lemma, dataset_imdb_labels)
}

In [ ]:
results_rf = []

# Перебор векторизаторов и датасетов
for vect_name, vect in [('Unigram', vect_uni), ('Bigram', vect_bi)]:
    for data_name, (data, labels) in all_datasets.items():
        print(f"Vectorizer: {vect_name}, dataset: {data_name}")

        # Векторизация (обучаем ТОЛЬКО на train данных в кросс-валидации)
        X = vect.fit_transform(data)

        # Поиск параметров
        grid_search_rf.fit(X, labels)

        # Лучшие параметры и метрики
        best_params = grid_search_rf.best_params_
        best_score = grid_search_rf.best_score_

        print(f"Best F1: {best_score:.4f}")
        print(f"Best params: {best_params}")
        print()

        # Сохраняем результаты
        results_rf.append({
            'Vectorizer': vect_name,
            'Dataset': data_name,
            'Best F1': best_score,
            'Best params': best_params
        })

# Результаты в DataFrame
results_df_rf = pd.DataFrame(results_rf)
print(results_df_rf)

# Обучаем LinearSVC
Сначала проведем обучение по стратифицированной кросс-валидации

In [ ]:
from sklearn.svm import LinearSVC
import pandas as pd
lin_scv = LinearSVC(C=1.0, random_state=42)  # параметры для SVM

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score

cv_strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # параметры для кросс-валидации

# определяем сетку параметров
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'class_weight': [None, 'balanced'],
    'loss': ['hinge', 'squared_hinge'],
    'penalty': ['l2', 'l1'],
    'dual': [True, False],
    'multi_class': ['ovr', 'crammer_singer'],
    'max_iter': [1000, 2000]
}

# объединяем всё в GridSearchCV
grid_search = GridSearchCV(lin_scv, param_grid, cv=cv_strat, scoring='f1_weighted', verbose=1, n_jobs=-1)

In [ ]:
# перебор всех датасетов

all_datasets ={
    'news_base': (dataset_news_base, dataset_news_labels),
    'news_stem': (dataset_news_stem, dataset_news_labels),
    'news_lemma': (dataset_news_lemma, dataset_news_labels),
    'imdb_base': (dataset_imdb_base, dataset_imdb_labels),
    'imdb_stem': (dataset_imdb_stem, dataset_imdb_labels),
    'imdb_lemma': (dataset_imdb_lemma, dataset_imdb_labels)
}

In [ ]:
results = []

# Перебор векторизаторов и датасетов
for vect_name, vect in [('Unigram', vect_uni), ('Bigram', vect_bi)]:
    for data_name, (data, labels) in all_datasets.items():
        print(f"Vectorizer: {vect_name}, dataset: {data_name}")

        # Векторизация (обучаем ТОЛЬКО на train данных в кросс-валидации)
        X = vect.fit_transform(data)

        # Поиск параметров
        grid_search.fit(X, labels)

        # Лучшие параметры и метрики
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_

        print(f"Best F1: {best_score:.4f}")
        print(f"Best params: {best_params}")
        print()

        # Сохраняем результаты
        results.append({
            'Vectorizer': vect_name,
            'Dataset': data_name,
            'Best F1': best_score,
            'Best params': best_params
        })

# Результаты в DataFrame
results_df = pd.DataFrame(results)
print(results_df)